# Mapping migration

Introduction to vector data operations

## STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r

:::

### Identify the ecoregion for each observation

You can combine the ecoregions and the observations **spatially** using
a method called `.sjoin()`, which stands for spatial join.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Check out the <a
href="https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins"><code>geopandas</code>
documentation on spatial joins</a> to help you figure this one out. You
can also ask your favorite LLM (Large-Language Model, like ChatGPT)</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Perform a spatial join</div></div><div class="callout-body-container callout-body"><p>Identify the correct values for the <code>how=</code> and
<code>predicate=</code> parameters of the spatial join.</p></div></div>

In [2]:
gbif_ecoregion_gdf = (
    ecoRegion_gdf
    # Match the CRS of the GBIF data and the ecoregions
    # Note: reproject the smaller dataset to avoid time suck
    .to_crs(gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf,
        how='inner', 
        predicate='contains')
        # select specific columns
        [['OBJECTID', 'gbifID', 'ECO_NAME','BIOME_NUM','BIOME_NAME', 'month', 'SHAPE_AREA']]
).reset_index()
gbif_ecoregion_gdf

,index,OBJECTID,gbifID,ECO_NAME,BIOME_NUM,BIOME_NAME,month,SHAPE_AREA
0,12,13.0,4621947377,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,5,17.133639
1,12,13.0,4743927038,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,5,17.133639
2,12,13.0,4761090115,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,6,17.133639
3,12,13.0,4765238615,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,7,17.133639
4,12,13.0,4630693711,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,6,17.133639
...,...,...,...,...,...,...,...,...
159532,839,845.0,4671409376,North Atlantic moist mixed forests,4.0,Temperate Broadleaf & Mixed Forests,9,5.586107
159533,839,845.0,4779728331,North Atlantic moist mixed forests,4.0,Temperate Broadleaf & Mixed Forests,9,5.586107
159534,839,845.0,4793070700,North Atlantic moist mixed forests,4.0,Temperate Broadleaf & Mixed Forests,9,5.586107
159535,839,845.0,4633848077,North Atlantic moist mixed forests,4.0,Temperate Broadleaf & Mixed Forests,10,5.586107


### Count the observations in each ecoregion each month

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Group observations by ecoregion</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>columns_to_group_by</code> with a list of columns.
Keep in mind that you will end up with one row for each group – you want
to count the observations in each ecoregion by month.</li>
<li>Select only month/ecosystem combinations that have more than one
occurrence recorded, since a single occurrence could be an error.</li>
<li>Use the <code>.groupby()</code> and <code>.mean()</code> methods to
compute the mean occurrences by ecoregion and by month.</li>
<li>Run the code – it will normalize the number of occurrences by month
and ecoretion.</li>
</ol></div></div>

In [3]:

# Raw occurrence data
occurrence_df = (
    gbif_ecoregion_gdf
    # For each ecoregion, for each month...
    # note that OBJECTID corresponds to the ecoregion
    .groupby(['OBJECTID', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('gbifID', 'count'))
)

# 350 rows
occurrence_df

# Get rid of rare observations (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]

# 308 rows
occurrence_df

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby('OBJECTID')
    .mean()
)

mean_occurrences_by_ecoregion

# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby('month')
    .mean()
)

mean_occurrences_by_month


,occurrences
month,
1,7.666667
2,7.333333
3,5.666667
4,140.472222
5,1207.169492
6,1041.571429
7,572.894737
8,164.972973
9,260.897436


### Normalize the observations

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Normalize</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion</li>
</ol></div></div>

In [4]:
# Normalize by space and time for sampling effort
occurrence_df['norm_occurrences'] = (
    occurrence_df
    / mean_occurrences_by_month
    / mean_occurrences_by_ecoregion
)
occurrence_df

occurrences  norm_occurrences
OBJECTID month                               
13.0     5                2          0.000828
         6                2          0.000960
         7                2          0.001746
17.0     4                2          0.000010
         5             2980          0.001732
...                     ...               ...
839.0    7              293          0.002173
         8               40          0.001030
         9               11          0.000179
845.0    9               25          0.005989
         10               7          0.013328

[308 rows x 2 columns]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Make sure to store the new version of your <code>DataFrame</code> for
other notebooks!</p>
<div id="f13606e9" class="cell" data-execution_count="9">
<div class="sourceCode" id="cb1"><pre
class="sourceCode python cell-code"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="op">%</span>store occurrence_df</span></code></pre></div>
</div></div></div>

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [5]:
%store occurrence_df

Stored 'occurrence_df' (DataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!